In [ ]:
import warnings
warnings.simplefilter("ignore")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
from datetime import datetime 
import time 
import csv
import os 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


School Characteristics NCES - school year 2018-2019

In [ ]:
url = "https://data-nces.opendata.arcgis.com/datasets/146f28635d01435e978c6d2d5ccf6ea9_0.csv?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D"
data = pd.read_csv(url, sep=",")

data = data[['NCESSCH', 'TITLEI', 'LEAID','G12','SCHOOL_LEVEL','TOTFRL','SCHOOL_TYPE_TEXT','FTE','STUTERATIO','TOTMENROL','TOTFENROL']].reset_index(drop=True)

Data is filtered to High school level. 3 new columns for percentage of male/female/free lunch.

In [ ]:
data = data.loc[data['SCHOOL_LEVEL'] =='High']
data["PCT_MEN"] = (abs(data['TOTMENROL'])/((abs(data['TOTMENROL']))+(abs(data['TOTFENROL']))))
data["PCT_FEN"] = (abs(data['TOTFENROL'])/((abs(data['TOTFENROL']))+(abs(data['TOTMENROL']))))
data["PCT_FRL"] = (abs(data['TOTFRL'])/((abs(data['TOTFENROL']))+(abs(data['TOTMENROL']))))
data.head(5)

,NCESSCH,TITLEI,LEAID,G12,SCHOOL_LEVEL,TOTFRL,SCHOOL_TYPE_TEXT,FTE,STUTERATIO,TOTMENROL,TOTFENROL,PCT_MEN,PCT_FEN,PCT_FRL
1,20061000470,1-Yes,200610,29.0,High,16,Alternative/other school,3.700000,11.1,25.0,16.0,0.609756,0.390244,0.390244
4,20039000513,1-Yes,200390,3.0,High,3,Alternative/other school,1.000000,9.0,7.0,2.0,0.777778,0.222222,0.333333
15,20072000661,1-Yes,200720,34.0,High,33,Regular school,14.400000,13.1,95.0,93.0,0.505319,0.494681,0.175532
18,20000100208,1-Yes,200001,61.0,High,393,Regular school,34.970001,14.1,259.0,235.0,0.524291,0.475709,0.795547
26,20000100318,1-Yes,200001,22.0,High,3,Alternative/other school,7.800000,7.7,30.0,30.0,0.500000,0.500000,0.050000


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23243 entries, 1 to 100676
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   NCESSCH           23243 non-null  int64  
 1   TITLEI            23243 non-null  object 
 2   LEAID             23243 non-null  int64  
 3   G12               23068 non-null  float64
 4   SCHOOL_LEVEL      23243 non-null  object 
 5   TOTFRL            23243 non-null  int64  
 6   SCHOOL_TYPE_TEXT  23243 non-null  object 
 7   FTE               22092 non-null  float64
 8   STUTERATIO        21432 non-null  float64
 9   TOTMENROL         23054 non-null  float64
 10  TOTFENROL         23030 non-null  float64
 11  PCT_MEN           21650 non-null  float64
 12  PCT_FEN           21650 non-null  float64
 13  PCT_FRL           22926 non-null  float64
dtypes: float64(8), int64(3), object(3)
memory usage: 2.7+ MB


In [ ]:
data['NCESSCH'].nunique()

23243

In [ ]:
data['TITLEI'] = data['TITLEI'].replace('\d+', '', regex=True)
data['TITLEI'] = data['TITLEI'].replace('-', '', regex=True)

In [ ]:
data['TITLEI'].unique()

array(['Yes', 'Not Applicable', 'Missing'], dtype=object)

School type map using dictionary - Regular is 1 and the others are 0

In [ ]:
SCHOOL_TYPE ={'Alternative/other school':0, 'Regular school':1, 'Vocational school':0,
       'Special education school':0}

data['SCHOOL_TYPE']=data['SCHOOL_TYPE_TEXT'].map(SCHOOL_TYPE)

data.drop(['SCHOOL_TYPE_TEXT'], axis=1, inplace=True)

Adjusted Graduation Rate database school year 2018-2019

In [ ]:
url2 = "https://www2.ed.gov/about/inits/ed/edfacts/data-files/acgr-sch-sy2018-19-wide.csv"
grad_rates = pd.read_csv(url2,sep=",")

New column "Success Rate" created using success list

In [ ]:
# removed the suffix from the columns for readability 
grad_rates.columns = grad_rates.columns.str.rstrip('_1819')

In [ ]:
# drop the category rates since we are interested in how the number of students in each category contributes to the overall success rate
grad_rates = grad_rates.drop(['MAM_RATE', 'MAS_RATE', 'MBL_RATE', 'MHI_RATE', 'MTR_RATE',
                             'MWH_RATE', 'CWD_RATE', 'ECD_RATE', 'FCS_RATE', 'HOM_RATE', 
                             'LEP_RATE'], axis=1)

In [ ]:
# fill all nulls with 0 because those are where the number of students for that category is just 0 and not null
grad_rates.fillna(0, inplace = True)

In [ ]:
grad_rates.head()

,STNAM,FIPST,LEAID,ST_LEAID,LEANM,NCESSCH,ST_SCHID,SCHNAM,ALL_COHORT,ALL_RATE,MAM_COHORT,MAS_COHORT,MBL_COHORT,MHI_COHORT,MTR_COHORT,MWH_COHORT,CWD_COHORT,ECD_COHORT,FCS_COHORT,HOM_COHORT,LEP_COHORT,DATE_CUR
0,ALABAMA,1,100005,AL-101,Albertville City,10000500871,AL-101-0020,Albertville High School,375,94,0.0,8.0,15.0,173.0,4.0,175.0,19.0,114.0,0.0,7.0,67.0,24JUL20
1,ALABAMA,1,100006,AL-048,Marshall County,10000600872,AL-048-0030,Asbury High School,55,GE90,0.0,0.0,0.0,24.0,0.0,31.0,7.0,15.0,0.0,3.0,3.0,24JUL20
2,ALABAMA,1,100006,AL-048,Marshall County,10000600878,AL-048-0100,Douglas High School,130,90-94,4.0,0.0,0.0,41.0,0.0,85.0,9.0,99.0,0.0,10.0,7.0,24JUL20
3,ALABAMA,1,100006,AL-048,Marshall County,10000600883,AL-048-0140,Kate D Smith DAR High School,97,GE95,0.0,0.0,1.0,1.0,0.0,95.0,4.0,34.0,1.0,2.0,1.0,24JUL20
4,ALABAMA,1,100006,AL-048,Marshall County,10000601585,AL-048-0042,Brindlee Mountain High School,64,85-89,0.0,0.0,0.0,4.0,1.0,59.0,7.0,47.0,0.0,7.0,1.0,24JUL20


In [ ]:
# split at hyphen and keep lower bound
grad_rates['ALL_RATE'] = grad_rates['ALL_RATE'].str.split('-').str[0]

In [ ]:
# remove any letters or punctuation from grad rate
grad_rates['ALL_RATE'] = grad_rates["ALL_RATE"].str.replace(r'\D', '')

In [ ]:
grad_rates['ALL_RATE'] = grad_rates["ALL_RATE"].apply(pd.to_numeric)

In [ ]:
grad_rates['Success_Rate'] = (grad_rates['ALL_RATE'] > 66).astype(int)

State ID, District ID, School ID, each Cohort, Success Rate are columns selected

In [ ]:
grad_rates = grad_rates[['STNAM', 'FIPST','LEAID','NCESSCH', 'ALL_RATE', 'ALL_COHORT','MAM_COHORT', 'MAS_COHORT', 'MBL_COHORT', 'MHI_COHORT', 'MTR_COHORT', 'MWH_COHORT', 'CWD_COHORT', 'ECD_COHORT', 'FCS_COHORT', 'HOM_COHORT', 'LEP_COHORT','Success_Rate']]


In [ ]:
grad_rates.head()

,STNAM,FIPST,LEAID,NCESSCH,ALL_RATE,ALL_COHORT,MAM_COHORT,MAS_COHORT,MBL_COHORT,MHI_COHORT,MTR_COHORT,MWH_COHORT,CWD_COHORT,ECD_COHORT,FCS_COHORT,HOM_COHORT,LEP_COHORT,Success_Rate
0,ALABAMA,1,100005,10000500871,94.0,375,0.0,8.0,15.0,173.0,4.0,175.0,19.0,114.0,0.0,7.0,67.0,1
1,ALABAMA,1,100006,10000600872,90.0,55,0.0,0.0,0.0,24.0,0.0,31.0,7.0,15.0,0.0,3.0,3.0,1
2,ALABAMA,1,100006,10000600878,90.0,130,4.0,0.0,0.0,41.0,0.0,85.0,9.0,99.0,0.0,10.0,7.0,1
3,ALABAMA,1,100006,10000600883,95.0,97,0.0,0.0,1.0,1.0,0.0,95.0,4.0,34.0,1.0,2.0,1.0,1
4,ALABAMA,1,100006,10000601585,85.0,64,0.0,0.0,0.0,4.0,1.0,59.0,7.0,47.0,0.0,7.0,1.0,1


In [ ]:
grad_rates['NCESSCH'].nunique()

22900

In [ ]:
grad_rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22900 entries, 0 to 22899
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   STNAM         22900 non-null  object 
 1   FIPST         22900 non-null  int64  
 2   LEAID         22900 non-null  int64  
 3   NCESSCH       22900 non-null  int64  
 4   ALL_RATE      21425 non-null  float64
 5   ALL_COHORT    22900 non-null  int64  
 6   MAM_COHORT    22900 non-null  float64
 7   MAS_COHORT    22900 non-null  float64
 8   MBL_COHORT    22900 non-null  float64
 9   MHI_COHORT    22900 non-null  float64
 10  MTR_COHORT    22900 non-null  float64
 11  MWH_COHORT    22900 non-null  float64
 12  CWD_COHORT    22900 non-null  float64
 13  ECD_COHORT    22900 non-null  float64
 14  FCS_COHORT    22900 non-null  float64
 15  HOM_COHORT    22900 non-null  float64
 16  LEP_COHORT    22900 non-null  float64
 17  Success_Rate  22900 non-null  int64  
dtypes: float64(12), int64(5), 

In [ ]:
# drop LEAID to merge on NCESSCH without making duplicates
grad_rates.drop('LEAID', axis=1, inplace=True)

Merge School Characteristics and Grad Rate tables

In [ ]:
Merged_data = pd.merge(data, grad_rates, left_on='NCESSCH', right_on='NCESSCH')

In [ ]:
Merged_data.head()

,NCESSCH,TITLEI,LEAID,G12,SCHOOL_LEVEL,TOTFRL,FTE,STUTERATIO,TOTMENROL,TOTFENROL,PCT_MEN,PCT_FEN,PCT_FRL,SCHOOL_TYPE,STNAM,FIPST,ALL_RATE,ALL_COHORT,MAM_COHORT,MAS_COHORT,MBL_COHORT,MHI_COHORT,MTR_COHORT,MWH_COHORT,CWD_COHORT,ECD_COHORT,FCS_COHORT,HOM_COHORT,LEP_COHORT,Success_Rate
0,20061000470,Yes,200610,29.0,High,16,3.700000,11.1,25.0,16.0,0.609756,0.390244,0.390244,0,ALASKA,2,21.0,16,11.0,1.0,0.0,1.0,2.0,1.0,2.0,3.0,0.0,0.0,6.0,0
1,20039000513,Yes,200390,3.0,High,3,1.000000,9.0,7.0,2.0,0.777778,0.222222,0.333333,0,ALASKA,2,50.0,8,2.0,0.0,0.0,0.0,0.0,6.0,3.0,0.0,0.0,0.0,0.0,0
2,20072000661,Yes,200720,34.0,High,33,14.400000,13.1,95.0,93.0,0.505319,0.494681,0.175532,1,ALASKA,2,90.0,35,5.0,16.0,0.0,5.0,0.0,9.0,1.0,8.0,0.0,0.0,5.0,1
3,20000100208,Yes,200001,61.0,High,393,34.970001,14.1,259.0,235.0,0.524291,0.475709,0.795547,1,ALASKA,2,55.0,85,72.0,2.0,0.0,3.0,0.0,8.0,14.0,49.0,3.0,0.0,18.0,0
4,20000100318,Yes,200001,22.0,High,3,7.800000,7.7,30.0,30.0,0.500000,0.500000,0.050000,0,ALASKA,2,21.0,23,23.0,0.0,0.0,0.0,0.0,0.0,4.0,20.0,1.0,1.0,14.0,0


In [ ]:
Merged_data['NCESSCH'].nunique()

19961

In [ ]:
Merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19961 entries, 0 to 19960
Data columns (total 30 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   NCESSCH       19961 non-null  int64  
 1   TITLEI        19961 non-null  object 
 2   LEAID         19961 non-null  int64  
 3   G12           19945 non-null  float64
 4   SCHOOL_LEVEL  19961 non-null  object 
 5   TOTFRL        19961 non-null  int64  
 6   FTE           19334 non-null  float64
 7   STUTERATIO    19141 non-null  float64
 8   TOTMENROL     19938 non-null  float64
 9   TOTFENROL     19936 non-null  float64
 10  PCT_MEN       19695 non-null  float64
 11  PCT_FEN       19695 non-null  float64
 12  PCT_FRL       19926 non-null  float64
 13  SCHOOL_TYPE   19961 non-null  int64  
 14  STNAM         19961 non-null  object 
 15  FIPST         19961 non-null  int64  
 16  ALL_RATE      19175 non-null  float64
 17  ALL_COHORT    19961 non-null  int64  
 18  MAM_COHORT    19961 non-nu

District ID needs leading 0 to be able to merge with financial data

In [ ]:
Merged_data['LEAID'] = Merged_data['LEAID'].map(lambda x: f'{x:0>7}')

Duplicate District ID is dropped

In [ ]:
Merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19961 entries, 0 to 19960
Data columns (total 30 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   NCESSCH       19961 non-null  int64  
 1   TITLEI        19961 non-null  object 
 2   LEAID         19961 non-null  object 
 3   G12           19945 non-null  float64
 4   SCHOOL_LEVEL  19961 non-null  object 
 5   TOTFRL        19961 non-null  int64  
 6   FTE           19334 non-null  float64
 7   STUTERATIO    19141 non-null  float64
 8   TOTMENROL     19938 non-null  float64
 9   TOTFENROL     19936 non-null  float64
 10  PCT_MEN       19695 non-null  float64
 11  PCT_FEN       19695 non-null  float64
 12  PCT_FRL       19926 non-null  float64
 13  SCHOOL_TYPE   19961 non-null  int64  
 14  STNAM         19961 non-null  object 
 15  FIPST         19961 non-null  int64  
 16  ALL_RATE      19175 non-null  float64
 17  ALL_COHORT    19961 non-null  int64  
 18  MAM_COHORT    19961 non-nu

Copy of data is made

In [ ]:
df = Merged_data.copy(deep=False)

Finance data for school year 2018-2019

In [ ]:
url4 = "https://www2.census.gov/programs-surveys/school-finances/tables/2019/secondary-education-finance/elsec19t.xls"
revenue = pd.read_excel(url4)

Dsitrict ID,  PER PUPIL - TOTAL CURRENT SPENDING (ELEMENTARY-SECONDARY) columns are selected

PER PUPIL SPENDING AMOUNTS
The per pupil spending amounts included in the summary tables and data files are derived from current spending totals and the fall membership data. Per pupil expenditure does not include spending for nonelementary-secondary programs (community service, adult education), or spending by a school system for students not included in its fall membership counts.

In [ ]:
revenue = revenue[['NCESID', 'PPCSTOT']]
revenue.shape

(14197, 2)

In [ ]:
revenue = revenue.rename(columns={"NCESID": "LEAID"})

In [ ]:
revenue['LEAID'].nunique()

14197

Merge revenue and other 2 tables

In [ ]:
All_data = pd.merge(Merged_data, revenue, left_on='LEAID', right_on='LEAID')

In [ ]:
All_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17855 entries, 0 to 17854
Data columns (total 31 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   NCESSCH       17855 non-null  int64  
 1   TITLEI        17855 non-null  object 
 2   LEAID         17855 non-null  object 
 3   G12           17841 non-null  float64
 4   SCHOOL_LEVEL  17855 non-null  object 
 5   TOTFRL        17855 non-null  int64  
 6   FTE           17470 non-null  float64
 7   STUTERATIO    17283 non-null  float64
 8   TOTMENROL     17834 non-null  float64
 9   TOTFENROL     17834 non-null  float64
 10  PCT_MEN       17600 non-null  float64
 11  PCT_FEN       17600 non-null  float64
 12  PCT_FRL       17825 non-null  float64
 13  SCHOOL_TYPE   17855 non-null  int64  
 14  STNAM         17855 non-null  object 
 15  FIPST         17855 non-null  int64  
 16  ALL_RATE      17128 non-null  float64
 17  ALL_COHORT    17855 non-null  int64  
 18  MAM_COHORT    17855 non-nu

Nulls dropped from merged data

In [ ]:
All_data.dropna()

,NCESSCH,TITLEI,LEAID,G12,SCHOOL_LEVEL,TOTFRL,FTE,STUTERATIO,TOTMENROL,TOTFENROL,PCT_MEN,PCT_FEN,PCT_FRL,SCHOOL_TYPE,STNAM,FIPST,ALL_RATE,ALL_COHORT,MAM_COHORT,MAS_COHORT,MBL_COHORT,MHI_COHORT,MTR_COHORT,MWH_COHORT,CWD_COHORT,ECD_COHORT,FCS_COHORT,HOM_COHORT,LEP_COHORT,Success_Rate,PPCSTOT
0,20061000470,Yes,0200610,29.0,High,16,3.700000,11.1,25.0,16.0,0.609756,0.390244,0.390244,0,ALASKA,2,21.0,16,11.0,1.0,0.0,1.0,2.0,1.0,2.0,3.0,0.0,0.0,6.0,0,36416
1,20061000286,Not Applicable,0200610,52.0,High,60,20.299999,11.9,130.0,111.0,0.539419,0.460581,0.248963,1,ALASKA,2,80.0,52,29.0,10.0,0.0,1.0,7.0,5.0,2.0,20.0,0.0,0.0,5.0,1,36416
2,20039000513,Yes,0200390,3.0,High,3,1.000000,9.0,7.0,2.0,0.777778,0.222222,0.333333,0,ALASKA,2,50.0,8,2.0,0.0,0.0,0.0,0.0,6.0,3.0,0.0,0.0,0.0,0.0,0,17859
3,20039000079,Yes,0200390,42.0,High,52,7.000000,9.9,36.0,33.0,0.521739,0.478261,0.753623,0,ALASKA,2,30.0,39,6.0,2.0,0.0,0.0,9.0,22.0,14.0,16.0,0.0,3.0,0.0,0,17859
4,20039000081,Yes,0200390,15.0,High,28,4.010000,8.5,18.0,16.0,0.529412,0.470588,0.823529,0,ALASKA,2,50.0,10,0.0,0.0,0.0,1.0,2.0,7.0,3.0,6.0,0.0,5.0,0.0,0,17859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17850,500041800540,Yes,5000418,22.0,High,109,23.100000,8.8,105.0,99.0,0.514706,0.485294,0.534314,1,VERMONT,50,80.0,27,0.0,1.0,0.0,0.0,1.0,25.0,11.0,16.0,0.0,0.0,0.0,1,16229
17851,500042500338,Yes,5000425,108.0,High,188,55.619999,11.3,340.0,287.0,0.542265,0.457735,0.299841,1,VERMONT,50,85.0,121,0.0,1.0,0.0,2.0,3.0,114.0,19.0,42.0,0.0,0.0,0.0,1,20224
17852,500042600543,Yes,5000426,40.0,High,58,30.150000,6.0,95.0,85.0,0.527778,0.472222,0.322222,1,VERMONT,50,80.0,50,0.0,0.0,0.0,0.0,0.0,49.0,6.0,28.0,0.0,0.0,0.0,1,16039
17853,500042700348,Yes,5000427,59.0,High,120,27.260000,11.7,146.0,172.0,0.459119,0.540881,0.377358,1,VERMONT,50,85.0,69,0.0,0.0,1.0,1.0,1.0,66.0,18.0,40.0,0.0,5.0,0.0,1,14916


Create new df with just NY and DMV data

In [ ]:
#ny_dmv = All_data.loc[(All_data['STNAM'] == 'DISTRICT OF COLUMBIA') | (All_data['STNAM'] == 'NEW YORK') | (All_data['STNAM'] == 'MARYLAND') | (All_data['STNAM'] == 'VIRGINIA')]

In [ ]:
#ny_dmv.drop(['STNAM', 'FIPST'], axis=1, inplace=True)

In [ ]:
#ny_dmv.info()

Assessment - Math & Reading 2018-2019

In [ ]:
Assessment = pd.read_csv('https://www2.ed.gov/about/inits/ed/edfacts/data-files/math-achievement-sch-sy2018-19-wide.csv')

KeyboardInterrupt: ignored

In [ ]:
Assessment = Assessment[['NCESSCH', 'ALL_MTHHSnumvalid_1819', 'ALL_MTHHSpctprof_1819']]

In [ ]:
Assessment.columns

Index(['NCESSCH', 'ALL_MTHHSnumvalid_1819', 'ALL_MTHHSpctprof_1819'], dtype='object')

In [ ]:
Assessment.columns = Assessment.columns.str.rstrip('_1819')

In [ ]:
column_list = Assessment.columns.to_list()
column_list

['NCESSCH', 'ALL_MTHHSnumvalid', 'ALL_MTHHSpctprof']

In [ ]:
Assessment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90278 entries, 0 to 90277
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   NCESSCH            90278 non-null  int64 
 1   ALL_MTHHSnumvalid  25285 non-null  object
 2   ALL_MTHHSpctprof   24597 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.1+ MB


In [ ]:
Assessment.head()

,NCESSCH,ALL_MTHHSnumvalid,ALL_MTHHSpctprof
0,10000500870,NaN,NaN
1,10000500871,317,39
2,10000500879,NaN,NaN
3,10000500889,NaN,NaN
4,10000600193,NaN,NaN


In [ ]:
# Nulls are elementary and middle school with no high school data
Assessment = Assessment.dropna()

In [ ]:
Assessment['NCESSCH'].nunique()

24597

In [ ]:
Assessment['ALL_MTHHSpctprof'] =Assessment['ALL_MTHHSpctprof'].str.split('-').str[0]
Assessment['ALL_MTHHSpctprof'] = Assessment["ALL_MTHHSpctprof"].str.replace(r'\D', '')
Assessment['ALL_MTHHSpctprof'] = Assessment["ALL_MTHHSpctprof"].apply(pd.to_numeric)
Assessment['ALL_MTHHSpctprof'] = Assessment['ALL_MTHHSpctprof'].fillna(Assessment['ALL_MTHHSpctprof'].mean())

In [ ]:
Assessment['ALL_MTHHSnumvalid'] = Assessment['ALL_MTHHSnumvalid'].str.split('.').str[0]
Assessment['ALL_MTHHSnumvalid'] = Assessment["ALL_MTHHSnumvalid"].apply(pd.to_numeric)
Assessment['ALL_MTHHSnumvalid'] = Assessment['ALL_MTHHSnumvalid'].fillna(Assessment['ALL_MTHHSpctprof'].mean())

Math Assessment data merged to other data

In [ ]:
all_math = pd.merge(All_data, Assessment, on="NCESSCH")

In [ ]:
all_math.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17341 entries, 0 to 17340
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   NCESSCH            17341 non-null  int64  
 1   TITLEI             17341 non-null  object 
 2   LEAID              17341 non-null  object 
 3   G12                17330 non-null  float64
 4   SCHOOL_LEVEL       17341 non-null  object 
 5   TOTFRL             17341 non-null  int64  
 6   FTE                17046 non-null  float64
 7   STUTERATIO         16935 non-null  float64
 8   TOTMENROL          17324 non-null  float64
 9   TOTFENROL          17325 non-null  float64
 10  PCT_MEN            17155 non-null  float64
 11  PCT_FEN            17155 non-null  float64
 12  PCT_FRL            17319 non-null  float64
 13  SCHOOL_TYPE        17341 non-null  int64  
 14  STNAM              17341 non-null  object 
 15  FIPST              17341 non-null  int64  
 16  ALL_RATE           168

Do the same for reading data 

In [ ]:
Assessment1 = pd.read_csv('https://www2.ed.gov/about/inits/ed/edfacts/data-files/rla-achievement-sch-sy2018-19-wide.csv')

In [ ]:
Assessment1.columns

Index(['STNAM', 'FIPST', 'LEANM', 'LEAID', 'ST_LEAID', 'NCESSCH', 'ST_SCHID',
       'SCHNAM', 'DATE_CUR', 'ALL_RLA00numvalid_1819',
       ...
       'MIL_RLA05numvalid_1819', 'MIL_RLA05pctprof_1819',
       'MIL_RLA06numvalid_1819', 'MIL_RLA06pctprof_1819',
       'MIL_RLA07numvalid_1819', 'MIL_RLA07pctprof_1819',
       'MIL_RLA08numvalid_1819', 'MIL_RLA08pctprof_1819',
       'MIL_RLAHSnumvalid_1819', 'MIL_RLAHSpctprof_1819'],
      dtype='object', length=265)

In [ ]:
Assessment1.columns = Assessment1.columns.str.rstrip('_1819')

In [ ]:
Assessment1 = Assessment1[['NCESSCH', 'ALL_RLAHSnumvalid', 'ALL_RLAHSpctprof']]

In [ ]:
Assessment1 = Assessment1.dropna()

In [ ]:
Assessment1['ALL_RLAHSpctprof'] = Assessment1['ALL_RLAHSpctprof'].str.split('-').str[0]
Assessment1['ALL_RLAHSpctprof'] = Assessment1['ALL_RLAHSpctprof'].str.replace(r'\D', '')
Assessment1['ALL_RLAHSpctprof'] = Assessment1['ALL_RLAHSpctprof'].apply(pd.to_numeric)
Assessment1['ALL_RLAHSpctprof'] = Assessment1['ALL_RLAHSpctprof'].fillna(Assessment1['ALL_RLAHSpctprof'].mean())

In [ ]:
Assessment1['ALL_RLAHSnumvalid'] = Assessment1['ALL_RLAHSnumvalid'].str.split('.').str[0]
Assessment1['ALL_RLAHSnumvalid'] = Assessment1['ALL_RLAHSnumvalid'].apply(pd.to_numeric)
Assessment1['ALL_RLAHSnumvalid'] = Assessment1['ALL_RLAHSnumvalid'].fillna(Assessment1['ALL_RLAHSpctprof'].mean())

In [ ]:
all_assessments = pd.merge(all_math, Assessment1, on="NCESSCH")

In [ ]:
all_assessments =all_assessments.rename(columns={"ALL_MTHHSnumvalid": "MATH_NUM", "ALL_MTHHSpctprof": "MATH_PCT", "ALL_RLAHSnumvalid": "READ_NUM", "ALL_RLAHSpctprof": "READ_PCT"})

In [ ]:
all_assessments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17174 entries, 0 to 17173
Data columns (total 35 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   NCESSCH       17174 non-null  int64  
 1   TITLEI        17174 non-null  object 
 2   LEAID         17174 non-null  object 
 3   G12           17163 non-null  float64
 4   SCHOOL_LEVEL  17174 non-null  object 
 5   TOTFRL        17174 non-null  int64  
 6   FTE           16889 non-null  float64
 7   STUTERATIO    16779 non-null  float64
 8   TOTMENROL     17157 non-null  float64
 9   TOTFENROL     17158 non-null  float64
 10  PCT_MEN       16989 non-null  float64
 11  PCT_FEN       16989 non-null  float64
 12  PCT_FRL       17152 non-null  float64
 13  SCHOOL_TYPE   17174 non-null  int64  
 14  STNAM         17174 non-null  object 
 15  FIPST         17174 non-null  int64  
 16  ALL_RATE      16678 non-null  float64
 17  ALL_COHORT    17174 non-null  int64  
 18  MAM_COHORT    17174 non-nu

In [ ]:
all_assessments['PCT_MEN'] = all_assessments['PCT_MEN'].fillna(all_assessments['PCT_MEN'].mean())
all_assessments['PCT_FEN'] = all_assessments['PCT_MEN'].fillna(all_assessments['PCT_FEN'].mean())
all_assessments['ALL_RATE'] = all_assessments['ALL_RATE'].fillna(all_assessments['ALL_RATE'].mean())
all_assessments['FTE'] = all_assessments['FTE'].fillna(all_assessments['FTE'].mean())

Created separate NY and DMV dataframe

In [ ]:
NY = all_assessments.loc[(all_assessments['STNAM'] == 'NEW YORK')]
NY.head()

,NCESSCH,TITLEI,LEAID,G12,SCHOOL_LEVEL,TOTFRL,FTE,STUTERATIO,TOTMENROL,TOTFENROL,PCT_MEN,PCT_FEN,PCT_FRL,SCHOOL_TYPE,STNAM,FIPST,ALL_RATE,ALL_COHORT,MAM_COHORT,MAS_COHORT,MBL_COHORT,MHI_COHORT,MTR_COHORT,MWH_COHORT,CWD_COHORT,ECD_COHORT,FCS_COHORT,HOM_COHORT,LEP_COHORT,Success_Rate,PPCSTOT,MATH_NUM,MATH_PCT,READ_NUM,READ_PCT
10560,361938005859,Not Applicable,3619380,84.0,High,85,27.020000,11.4,133.0,175.0,0.431818,0.431818,0.275974,1,NEW YORK,36,95.0,78,0.0,1.0,2.0,7.0,3.0,65.0,12.0,25.0,0.0,0.0,0.0,1,28407,40.504172,95.0,50.594066,95.0
10561,360585005857,Yes,3605850,135.0,High,585,51.009998,14.3,364.0,366.0,0.498630,0.498630,0.801370,1,NEW YORK,36,50.0,152,3.0,7.0,123.0,12.0,2.0,5.0,34.0,121.0,0.0,5.0,11.0,0,21573,40.504172,70.0,50.594066,70.0
10562,360585000297,Yes,3605850,120.0,High,434,67.019997,10.8,219.0,505.0,0.302486,0.302486,0.599448,1,NEW YORK,36,90.0,102,1.0,1.0,63.0,13.0,2.0,22.0,11.0,53.0,0.0,6.0,0.0,1,21573,40.504172,75.0,50.594066,80.0
10563,360585000301,Yes,3605850,110.0,High,396,47.009998,10.5,331.0,163.0,0.670040,0.670040,0.801619,0,NEW YORK,36,45.0,143,1.0,7.0,107.0,14.0,3.0,11.0,32.0,111.0,0.0,7.0,5.0,0,21573,40.504172,80.0,50.594066,65.0
10564,360585000307,Yes,3605850,97.0,High,400,47.000000,10.2,160.0,320.0,0.333333,0.333333,0.833333,1,NEW YORK,36,80.0,104,0.0,1.0,66.0,14.0,2.0,21.0,27.0,78.0,0.0,2.0,2.0,1,21573,40.504172,85.0,50.594066,85.0


In [ ]:
NY.shape

(630, 35)

In [ ]:
NY.isna().sum()

NCESSCH         0
TITLEI          0
LEAID           0
G12             0
SCHOOL_LEVEL    0
TOTFRL          0
FTE             0
STUTERATIO      2
TOTMENROL       0
TOTFENROL       0
PCT_MEN         0
PCT_FEN         0
PCT_FRL         0
SCHOOL_TYPE     0
STNAM           0
FIPST           0
ALL_RATE        0
ALL_COHORT      0
MAM_COHORT      0
MAS_COHORT      0
MBL_COHORT      0
MHI_COHORT      0
MTR_COHORT      0
MWH_COHORT      0
CWD_COHORT      0
ECD_COHORT      0
FCS_COHORT      0
HOM_COHORT      0
LEP_COHORT      0
Success_Rate    0
PPCSTOT         0
MATH_NUM        0
MATH_PCT        0
READ_NUM        0
READ_PCT        0
dtype: int64

In [ ]:
DMV = all_assessments.loc[(all_assessments['STNAM'] == 'MARYLAND') | (all_assessments['STNAM'] == 'DISTRICT OF COLUMBIA') | (all_assessments['STNAM'] == 'VIRGINIA')]
DMV.head()

,NCESSCH,TITLEI,LEAID,G12,SCHOOL_LEVEL,TOTFRL,FTE,STUTERATIO,TOTMENROL,TOTFENROL,PCT_MEN,PCT_FEN,PCT_FRL,SCHOOL_TYPE,STNAM,FIPST,ALL_RATE,ALL_COHORT,MAM_COHORT,MAS_COHORT,MBL_COHORT,MHI_COHORT,MTR_COHORT,MWH_COHORT,CWD_COHORT,ECD_COHORT,FCS_COHORT,HOM_COHORT,LEP_COHORT,Success_Rate,PPCSTOT,MATH_NUM,MATH_PCT,READ_NUM,READ_PCT
3268,110003000264,Not Applicable,1100030,85.0,High,-1,21.74,19.6,254.0,172.0,0.596244,0.596244,0.002347,0,DISTRICT OF COLUMBIA,11,6.0,169,0.0,0.0,80.0,88.0,1.0,0.0,36.0,158.0,9.0,21.0,77.0,0,22406,24.0,20.0,19.0,20.0
3269,110003000267,Not Applicable,1100030,139.0,High,-1,43.50,14.6,267.0,366.0,0.421801,0.421801,0.001580,1,DISTRICT OF COLUMBIA,11,95.0,143,2.0,3.0,122.0,14.0,2.0,0.0,9.0,57.0,2.0,6.0,5.0,1,22406,185.0,30.0,171.0,65.0
3270,110003000008,Not Applicable,1100030,112.0,High,-1,60.00,10.8,375.0,270.0,0.581395,0.581395,0.001550,1,DISTRICT OF COLUMBIA,11,60.0,164,0.0,1.0,84.0,77.0,1.0,1.0,29.0,126.0,9.0,26.0,59.0,0,22406,139.0,5.0,136.0,6.0
3271,110003000016,Not Applicable,1100030,110.0,High,-1,4.00,136.3,171.0,374.0,0.313761,0.313761,0.001835,1,DISTRICT OF COLUMBIA,11,95.0,114,0.0,4.0,86.0,13.0,2.0,9.0,15.0,44.0,4.0,5.0,1.0,1,22406,145.0,15.0,141.0,55.0
3272,110003000055,Not Applicable,1100030,136.0,High,-1,47.00,9.7,268.0,187.0,0.589011,0.589011,0.002198,1,DISTRICT OF COLUMBIA,11,65.0,180,0.0,0.0,174.0,6.0,0.0,0.0,36.0,137.0,12.0,15.0,1.0,0,22406,98.0,6.0,94.0,15.0


In [ ]:
DMV.shape

(571, 35)

Create df for national data

In [ ]:
national_data = all_assessments

In [ ]:
national_data.isna().sum()

NCESSCH           0
TITLEI            0
LEAID             0
G12              11
SCHOOL_LEVEL      0
TOTFRL            0
FTE               0
STUTERATIO      395
TOTMENROL        17
TOTFENROL        16
PCT_MEN           0
PCT_FEN           0
PCT_FRL          22
SCHOOL_TYPE       0
STNAM             0
FIPST             0
ALL_RATE          0
ALL_COHORT        0
MAM_COHORT        0
MAS_COHORT        0
MBL_COHORT        0
MHI_COHORT        0
MTR_COHORT        0
MWH_COHORT        0
CWD_COHORT        0
ECD_COHORT        0
FCS_COHORT        0
HOM_COHORT        0
LEP_COHORT        0
Success_Rate      0
PPCSTOT           0
MATH_NUM          0
MATH_PCT          0
READ_NUM          0
READ_PCT          0
dtype: int64

Create NY csv

In [ ]:
NY.to_csv("/content/drive/Shareddrives/DATA 602 Final Project/NY_GRAD_FINANCE_RACE_CLEAN.csv")

Create DMV

In [ ]:
DMV.to_csv("/content/drive/Shareddrives/DATA 602 Final Project/DMV_GRAD_FINANCE_RACE_CLEAN.csv")

Create National Data cav 

In [ ]:
national_data.to_csv("/content/drive/Shareddrives/DATA 602 Final Project/NATIONAL_GRAD_FINANCE_RACE_CLEAN.csv")